In [1]:
import pandas as pd
from sklearn.utils import shuffle
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import os
from skimage import io
import torch
from torchvision import transforms
import torchvision
from skimage import color
import copy

import time
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models
from torchvision import transforms as T
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pandas as pd
from skimage import io, transform
import matplotlib.image as mpimg
from PIL import Image
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F
import scipy
import random
import pickle
import scipy.io as sio
import itertools
from scipy.ndimage.interpolation import shift
import copy
import warnings
#warnings.filterwarnings("ignore")
plt.ion()
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

## 1. Prepare Train/Valid/Test Set

In [4]:
os.getcwd()

'/Users/nhungle/Box/Free/Data-Science-Projects/Breast_Cancer_Diagnosis'

In [127]:
excel_path = '/Users/nhungle/Box/Free/Data-Science-Projects/Breast_Cancer_Diagnosis/excel_files'

In [128]:
cal_test = pd.read_csv(os.path.join(excel_path, 'calc_case_description_test_set.csv'))
cal_train = pd.read_csv(os.path.join(excel_path, 'calc_case_description_train_set.csv'))
mass_test = pd.read_csv(os.path.join(excel_path, 'mass_case_description_test_set.csv'))
mass_train = pd.read_csv(os.path.join(excel_path, 'mass_case_description_train_set.csv'))

In [134]:
cal_test = cal_test[['patient_id','left or right breast', 'image view',
            'pathology', 'image file path',
        'cropped image file path', 'ROI mask file path']]
mass_test = mass_test[['patient_id','left or right breast', 'image view',
            'pathology', 'image file path',
        'cropped image file path', 'ROI mask file path']]
cal_train = cal_train[['patient_id','left or right breast', 'image view',
        'pathology', 'image file path',
    'cropped image file path', 'ROI mask file path']]
mass_train = mass_train[['patient_id','left or right breast', 'image view',
        'pathology', 'image file path',
    'cropped image file path', 'ROI mask file path']]

In [10]:
cal_test.dropna(axis = 0, how = 'all', inplace = True)
mass_test.dropna(axis = 0, how = 'all', inplace = True)
testSet = pd.concat([cal_test, mass_test], axis=0)
train_valid = pd.concat([cal_train, mass_train], axis=0)

In [11]:
train_valid['class'] = 0
train_valid.loc[train_valid['pathology'] == 'MALIGNANT', 'class'] = 1
testSet['class'] = 0
testSet.loc[testSet['pathology'] == 'MALIGNANT', 'class'] = 

In [13]:
patient_list = list(train_valid['patient_id'].unique())
print('Number of patients: {}'.format(len(patient_list)))

Number of patients: 1248


In [14]:
unique_id_leftright = train_valid.groupby(['patient_id', 'left or right breast']).size().reset_index().rename(columns={0:'count'})
unique_id_leftright = shuffle(unique_id_leftright)
train_size = int(len(unique_id_leftright) * 0.8)
#print(train_size)
train_id_leftright = unique_id_leftright[:train_size][['patient_id', 'left or right breast']]
valid_id_leftright = unique_id_leftright[train_size:][['patient_id', 'left or right breast']]

In [15]:
trainSet = train_valid[(train_valid['patient_id'].isin(train_id_leftright['patient_id'])) &
            (train_valid['left or right breast'].isin(train_id_leftright['left or right breast'])) ]

In [16]:
validSet = train_valid[(train_valid['patient_id'].isin(valid_id_leftright['patient_id'])) &
            (train_valid['left or right breast'].isin(valid_id_leftright['left or right breast'])) ]

In [58]:
# Write to CSV
trainSet.to_csv(os.path.join(excel_path, 'twoClass_trainSet.csv'))
validSet.to_csv(os.path.join(excel_path, 'twoClass_validSet.csv'))
testSet.to_csv(os.path.join(excel_path, 'twoClass_testSet.csv'))

In [152]:
# Get a subset of data 
trainSet_sample = trainSet[:500]
valSet_sample = validSet[:100]
testSet_sample = testSet[:100]

In [154]:
trainSet_sample.to_csv(os.path.join(excel_path, 'twoClass_trainSet_sample.csv'))
valSet_sample.to_csv(os.path.join(excel_path, 'twoClass_validSet_sample.csv'))
testSet_sample.to_csv(os.path.join(excel_path, 'twoClass_testSet_sample.csv'))

### Prepare train/validatin/test local set to run code locally

In [53]:
def getImageName(image_string): 
    return image_string.split('/')[3].split('-')[-1] + '.dcm'

In [70]:
trainSet['local_image'] = ''
trainSet['local_image'] = trainSet['image file path'].apply(lambda x: getImageName(x))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [115]:
images_files = ['00446.dcm', '32980.dcm', '38548.dcm', '44292.dcm', '64189.dcm', '64555.dcm', 
         '68297.dcm', '83789.dcm', '96727.dcm', '97907.dcm']
local_frame = trainSet[trainSet['local_image'].isin(images_files)]

local_frame

,patient_id,left or right breast,image view,pathology,image file path,cropped image file path,ROI mask file path,class,local_image
0,P_00005,RIGHT,CC,MALIGNANT,CBIS-DDSM/Calc-Training_P_00005_RIGHT_CC/08-07...,CBIS-DDSM/Calc-Training_P_00005_RIGHT_CC_1/08-...,CBIS-DDSM/Calc-Training_P_00005_RIGHT_CC_1/08-...,1,38548.dcm
1,P_00005,RIGHT,MLO,MALIGNANT,CBIS-DDSM/Calc-Training_P_00005_RIGHT_MLO/08-0...,CBIS-DDSM/Calc-Training_P_00005_RIGHT_MLO_1/09...,CBIS-DDSM/Calc-Training_P_00005_RIGHT_MLO_1/09...,1,96727.dcm
2,P_00007,LEFT,CC,BENIGN,CBIS-DDSM/Calc-Training_P_00007_LEFT_CC/08-07-...,CBIS-DDSM/Calc-Training_P_00007_LEFT_CC_1/09-0...,CBIS-DDSM/Calc-Training_P_00007_LEFT_CC_1/09-0...,0,68297.dcm
3,P_00007,LEFT,MLO,BENIGN,CBIS-DDSM/Calc-Training_P_00007_LEFT_MLO/08-07...,CBIS-DDSM/Calc-Training_P_00007_LEFT_MLO_1/09-...,CBIS-DDSM/Calc-Training_P_00007_LEFT_MLO_1/09-...,0,00446.dcm
5,P_00008,LEFT,MLO,BENIGN_WITHOUT_CALLBACK,CBIS-DDSM/Calc-Training_P_00008_LEFT_MLO/08-07...,CBIS-DDSM/Calc-Training_P_00008_LEFT_MLO_1/09-...,CBIS-DDSM/Calc-Training_P_00008_LEFT_MLO_1/09-...,0,97907.dcm
7,P_00008,RIGHT,MLO,BENIGN_WITHOUT_CALLBACK,CBIS-DDSM/Calc-Training_P_00008_RIGHT_MLO/08-0...,CBIS-DDSM/Calc-Training_P_00008_RIGHT_MLO_1/09...,CBIS-DDSM/Calc-Training_P_00008_RIGHT_MLO_1/09...,0,32980.dcm


In [116]:
train_local = local_frame[:4]
validation_local = local_frame[4:5]
test_local = local_frame[5:6]

In [117]:
train_local.to_csv(os.path.join(excel_path, 'train_local.csv'))
validation_local.to_csv(os.path.join(excel_path, 'validation_local.csv'))
test_local.to_csv(os.path.join(excel_path, 'test_local.csv'))

In [118]:
train_temp = pd.read_csv(os.path.join(excel_path, 'train_local.csv'))
train_temp.shape

(4, 10)